In [71]:
import os
import numpy as np 
import astropy.units as u
from desisim.io import read_basis_templates
from scipy.spatial import cKDTree as KDTree
from desisim.io import empty_metatable
from desisim.templates import GALAXY
import desisim.simexp
from specsim.simulator import Simulator

# desired redshift, r-band magnitude, g-r color (arbitrary blue galaxy in this case), and fiber loss fraction
z_desired = 0.2
rmag_desired = 16.0
gr_color_desired = 0.5
fibloss_desired = 0.1


In [72]:
# read meta data of templates 
meta_template = read_basis_templates(objtype='BGS', onlymeta=True)

# compile properties you want to match by for the template. This can be adjusted to whichever properties you want.
template_prop = np.vstack((
    meta_template['Z'].data, # redshift
    meta_template['SDSS_UGRIZ'].data[:,2], # r-band mag
    meta_template['SDSS_UGRIZ'].data[:,1] - meta_template['SDSS_UGRIZ'].data[:,2] # g-r color
)).T


INFO:io.py:951:read_basis_templates: Reading /global/cfs/cdirs/desi/spectro/templates/basis_templates/v3.2/bgs_templates_v2.3.fits metadata.


In [73]:
# match to template using KDTree
tree = KDTree(template_prop)

# get matched template ID
_, match_temp_id = tree.query(np.vstack([z_desired, rmag_desired, gr_color_desired]).T)
assert match_temp_id < len(meta_template)

In [74]:
#ls)
Source = GALAXY(objtype='BGS', minwave=3523, maxwave=9923., normline=None)

input_meta = empty_metatable(nmodel=1, objtype='BGS', input_meta=True)
input_meta['SEED']       = np.random.randint(2**32, size=1) 
input_meta['MAG']        = rmag_desired # r band apparent magnitude
input_meta['MAGFILTER']  = 'decam2014-r'
input_meta['REDSHIFT']   = z_desired # redshift
input_meta['VDISP']      = 100 
input_meta['TEMPLATEID'] = match_temp_id

source_flux, source_wave, source_meta, source_objmeta = Source.make_galaxy_templates(nmodel=1, input_meta=input_meta)

INFO:io.py:971:read_basis_templates: Reading /global/cfs/cdirs/desi/spectro/templates/basis_templates/v3.2/bgs_templates_v2.3.fits


In [75]:
# get specsim configuration 
config = desisim.simexp._specsim_config_for_wave(source_wave, dwave_out=0.8, specsim_config_file='desi')

In [76]:
# get nominal sky brightness
nominal_surface_brightness_dict = config.load_table(config.atmosphere.sky, 'surface_brightness', as_dict=True)
nominal_dark_sky_brightness = nominal_surface_brightness_dict['dark']

In [77]:
# initiate Simulator object
desi = Simulator(config, num_fibers=1)

In [78]:
# specify the exposure time
desi.observation.exposure_time = 180. * u.s
# specify the sky brightness
desi.atmosphere._surface_brightness_dict[desi.atmosphere.condition] = nominal_dark_sky_brightness
desi.atmosphere._extinct_emission = False
desi.atmosphere._moon = None 
desi.atmosphere.airmass = 1.2

In [79]:
# simulate
desi.simulate(
    source_fluxes=source_flux * 1e-17 * desi.simulated['source_flux'].unit, 
    fiber_acceptance_fraction=np.tile(fibloss_desired, source_flux.shape) # fiber-loss
)

random_state = np.random.RandomState(0)
desi.generate_random_noise(random_state, use_poisson=True)

waves, fluxes, ivars, ivars_electron = [], [], [], [] 
for table in desi.camera_output:
    _wave = table['wavelength'].astype(float)
    _flux = (table['observed_flux'] + table['random_noise_electrons'] * table['flux_calibration']).T.astype(float)
    _flux = _flux * 1e17

    _ivar = table['flux_inverse_variance'].T.astype(float)
    _ivar = _ivar / 1e34

    waves.append(_wave)
    fluxes.append(_flux)
    ivars.append(_ivar)

In [80]:
for band, flux, ivar in zip(['b', 'r', 'z'], fluxes, ivars): 
    print('%s SNR = %2f' % (band, np.median(flux * np.sqrt(ivar))))

b SNR = 3.779864
r SNR = 9.960613
z SNR = 12.403721
